# AI MAKERSPACE . COHORT 2 . MIDETERM

This is the Meta Financial Assistant Challenge

It's a RAG based Q&A application with a simple web interface

### [ CONFIGURATION OVERVIEW ]
- Data:Meta 10-k Filings (PDF file)
- LLM: OpenAI GPT-3.5-turbo
- Embedding Model: text-3-embedding small
- Infrastructure: LangChain
- Vector Store: Qdrant
- Deployment: Chainlit, Hugging Face

### [ DEVELOPMENT PHASES ]
1) Deploy a Hugging Face Space with a simple Chainlit Q&A App using GPT-3

2) RAG (Data, Embeddings, Vector Store, Langchain, Prompt Template)

3) Answer the 2 Questions (Test, Tweek, Repeat)

4) RAGAS some metric (Maybe)

### [ RAG TASKS ]
    Task 1: Load Dependencies & Setup
    Task 2: Setup the RAG Pipeline
    Task 3: Test Drive Questions
    Task 4: Adjust to Improve
    Task 5: Ready to Chat - make it an App

### Step 1: Dependencies and Initialization

The basic setup and getting the OpenAI key to use their LLM and embeddings model.

In [1]:
!pip install -qU langchain langchain-core langchain-community langchain-openai

In [2]:
!pip install -qU qdrant-client

In [3]:
!pip install -qU tiktoken pymupdf

In [4]:
import os
import getpass

os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key:")

OpenAI API Key:··········


### Step 2: Get the Data - PDF file on the Internet
In this challenge, we will be creating a RAG application that will use the Meta 10-k Filings.  This is a long 140 page financial document, chocked full of hidden data gems.  To start will grab the doc off the internet.

In [5]:
from langchain.document_loaders import PyMuPDFLoader

docs = PyMuPDFLoader("https://d18rn0p25nwr6d.cloudfront.net/CIK-0001326801/c7318154-f6ae-4866-89fa-f0c589f2ee3d.pdf").load()

### Step 3: Chunking the document
Chunking is breaking the document up into blocks of text that will be use by the retriever in our RAG pipeline.  We will be using the recursive character text splitter with a chunk size of 200, and no overlap.  Research for this type of app suggested a size of 500 and an overap of 50, but that did not fair well in answering specific questions.

In [6]:
import tiktoken

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-3.5-turbo").encode(
        text,
    )
    return len(tokens)

In [7]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 200,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_documents(docs)

### c) Step 4: Store chunks in a Qdrant vector storage using OpenAI embeddings
Qdrant, the vector database will be used.  It will be configured to use memory and will have a name of MetaFin.  LangChain has simple integration.

In [8]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [9]:
from langchain_community.vectorstores import Qdrant

qdrant_vectorstore = Qdrant.from_documents(
    split_chunks,
    embedding_model,
    location=":memory:",
    collection_name="MetaFin",
)

### Step 5: Set up Retriever
Very easy to set up the retriever.

In [10]:
qdrant_retriever = qdrant_vectorstore.as_retriever()

### Step 6: Setup LangChain RAG pipeline using OpenAI 3.5 turbo LLM
We will be using Open AI 3.5 turbo as the LLM.  The prompt template is simple and provides basic guidance on what is expected as a financial advisor.  The heart of this RAG pipeline is the LangChain:
[query]->[retriever]->[prompt template]->[LLM]->[inference]
which can easily be integrated into a chatbot web application.


In [18]:
from langchain_openai import ChatOpenAI

openai_chat_model = ChatOpenAI(model="gpt-3.5-turbo")

In [28]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """
CONTEXT:
{context}

QUERY:
{question}

Use the provided context to answer the user's query. You are a professional financial expert. You always review the provided financial information.  You provide correct, substantiated answers. You may not answer the user's query unless there is a specific context in the following text. If asked about the Board of Directors, then add Mark Zuckerberg as the "Board Chair".
If you do not know the answer, or cannot answer, please respond with "Insufficient data for further analysis, please try again". >>
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [29]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough

retrieval_augmented_qa_chain = (
    {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
    | RunnablePassthrough.assign(context=itemgetter("context"))
    | {"response": rag_prompt | openai_chat_model, "context": itemgetter("context")}
)

### Q & A Ready
OK. Lets try some questions.

In [30]:
chainlit_question = "What was the total value of 'Cash and cash equivalents' as of December 31, 2023?"
response = retrieval_augmented_qa_chain.invoke({"question" : chainlit_question })

In [31]:
chainlin_answer = response["response"].content
print(chainlin_answer)

The total value of 'Cash and cash equivalents' as of December 31, 2023, was $65.40 billion. This information is provided in the document within the context.


In [32]:
chainlit_question = "Who are Meta's 'Directors' (i.e., members of the Board of Directors)?"
response = retrieval_augmented_qa_chain.invoke({"question" : chainlit_question })

In [33]:
chainlin_answer = response["response"].content
print(chainlin_answer)

The Directors (members of the Board of Directors) of Meta as mentioned in the provided context are:

1. Peggy Alford
2. Marc L. Andreessen
3. Andrew W. Houston
4. Nancy Killefer
5. Robert M. Kimmitt
6. Sheryl K. Sandberg
7. Tracey T. Travis
8. Tony Xu

Additionally, Mark Zuckerberg serves as the Board Chair of Meta.

Therefore, the Board of Directors of Meta consists of the aforementioned individuals.


In [24]:
for context in response["context"]:
  print(context)

page_content="(State or other jurisdiction of incorporation or organization)\n(I.R.S. Employer Identification Number)\n1 Meta Way, Menlo Park, California 94025\n(Address of principal executive offices and Zip Code)\n(650)\xa0543-4800\n(Registrant's telephone number, including area code)\n__________________________\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each class\nTrading symbol(s)\nName of each exchange on which registered\nClass A Common Stock, $0.000006 par value\nMETA\nThe Nasdaq Stock Market LLC\nSecurities registered pursuant to Section 12(g) of the Act: None\nIndicate by check mark if the registrant is a well-known seasoned issuer, as defined in Rule 405 of the Securities Act.\xa0\xa0\xa0\xa0Yes\xa0\xa0☒\xa0\xa0No\xa0\xa0 ☐" metadata={'source': 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001326801/c7318154-f6ae-4866-89fa-f0c589f2ee3d.pdf', 'file_path': 'https://d18rn0p25nwr6d.cloudfront.net/CIK-0001326801/c7318154-f6ae-4866-89fa-f0c589f2ee3d.pdf',

In [34]:
response = retrieval_augmented_qa_chain.invoke({"question" : "What is the purpose of Meta?"})

In [35]:
chainlin_answer = response["response"].content
print(chainlin_answer)

The purpose of Meta is to enable a range of social experiences through virtual and augmented reality technologies, allowing people to engage in gaming, fitness, entertainment, and more. Additionally, Meta is committed to providing a skilled, inclusive, and diverse workforce to fuel innovation and growth, as well as offering benefits and resources to help employees thrive.


In [38]:
response = retrieval_augmented_qa_chain.invoke({"question" : "Who invented the Facebook thumbs up?"})

In [39]:
chainlin_answer = response["response"].content
print(chainlin_answer)

Insufficient data for further analysis, please try again
